In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import serial
import time
import keyboard
from IPython.display import clear_output

In [22]:
# connect to Arduino

ser = serial.Serial('COM3', 9600, timeout=1) #check whether the Arduino is really connected with COM3 or adjust the code to the correct COM
                                             #check at settings and check whether Arduino is a connected device
time.sleep(2)

In [23]:
# read write the serial connection at a given sampling rate

def write_read(x):                  #this function takes an input x, which is a switch command and it returns data from the Arduino
    ser.write(bytes(x, 'utf-8'))    #pass the command to Arduino by the serial connection
    data = ser.readline()           #take data from Arduino
    return data                     #return the data to be processed further

In [24]:
values = []
timestamp = []

for j in range(0,2):
    num = '1'                           #power supply = True
    switchtime = time.time()

    for i in range(0,10):
        timestamp.append(time.time()- switchtime)
        clear_output(wait=True)
        value = write_read(num)
        dummy = value.decode().rstrip().split()                                     # transform the byte values that came in...
        values.append(dummy)
        #dum0 = np.array([[int(j)/1024*5 for j in dummy[1:]]])
        print(pd.to_datetime(time.time(), unit= 's'),dummy, 
        #dum0[0],
              '            ', end = "\r")
    
    num = '0'
    switchtime = time.time()
    
    for i in range(0,10):
        timestamp.append(time.time()- switchtime)
        clear_output(wait=True)
        value = write_read(num)
        dummy = value.decode().rstrip().split()                                     # transform the byte values that came in...
        values.append(dummy)
        # dum0 = np.array([[int(j)/1024*5 for j in dummy[1:]]])
        print(pd.to_datetime(time.time(), unit= 's'),value, 
        #dum0[0],
              '            ', end = "\r")
    
    
ser.close()       #close the serial port when the reading is stopped. If not done, reconnection with the Arduino is not possible    

In [31]:
df = pd.DataFrame(values, 
                  index = timestamp,
                  columns = ['S', 'V0_measured', 'V1_measured', 'V2_measured'])
df.iloc[:,1:] = df.iloc[:,1:].astype(int).div(1024/5)

In [32]:
#df.to_excel('kanweg.xlsx')

In [33]:
df.iloc[:30,:]

,S,V0_measured,V1_measured,V2_measured
0.000000,1,4.995117,2.504883,0.0
1.025195,1,4.990234,2.885742,0.766602
2.051423,1,4.990234,3.203125,1.40625
3.087143,1,4.990234,3.476562,1.948242
4.120090,1,4.990234,3.706055,2.407227
0.000000,0,0.0,1.391602,2.792969
1.027989,0,0.0,1.171875,2.348633
2.047383,0,0.0,0.991211,1.99707
3.077499,0,0.0,0.834961,1.68457
4.110709,0,0.0,0.712891,1.430664
